# Dark Web Data Analysis

by Andy Alba

## Packages Used 

In [1]:
import pandas as pd
import numpy as np
import re
from os import listdir
import time


import seaborn as sns
import matplotlib.pyplot as plt

import plotly
import plotly.plotly as py
import plotly.graph_objs as go

## Classification Algorithim

In [3]:
def get_agora_folders():
    dirs = listdir("../data/grams")
    try:
        dirs.remove('.DS_Store')
    except:
       None 

    dirs = ["../data/grams/" + date for date in dirs if date != '2014-06-11-2']
    absolute_files = [date + "/" + i for date in dirs for i in listdir(date) if i == 'Agora.csv']
    absolute_files.sort()
    return absolute_files

agora_files = get_agora_folders()

def make_df_for_market(path):
    """
        path is the path to the agora csv. 
        returns the dated csv of information.
    """
    df = pd.read_csv(path)
    df['date'] = path
    df = df.drop(['hash', 'market_name','item_link', 'image_link', 'add_time', 'Unnamed: 10'], axis = 1)
    df['date'] = df['date'].str.replace("[^(\d+\-\d+\-\d+)]", "")
    df['date'] = pd.to_datetime(df['date'])
    df.columns = ['Vendor', 'Price', 'Item', 'Item Description', 'Origin', 'Date']
    df['Category'] = None
    df = df[['Category', 'Vendor', 'Item', 'Item Description', 'Price', 'Origin', 'Date']]
    df = df.drop_duplicates() 
    return df

In [5]:
# These First 4 are for organzining Grams Data

drug_bank = ["(\d*g)", "(\d\s*g)", "(\d*g)", "(\d\s*g)",
             "(\d\s*ml)", "(\d*ml)", "(\d*ml)", "(\d\s*ml)",
              "(\d\s*mcu)", "(\d*mcu)", "(\d*mcu)", "(\d\s*mcu)", 'seed',
              "(\d\s*oz)", "(\d*oz)", "(\d*oz)", "(\d\s*oz)", "cannabis"
              "(\d\s*ounce)", "(\d*ounce)", "(\d*ounce)", "(\d\s*ounce)",
              "tab", 'crack', 'xtc', 'weed', 'diesel', 'kilo', 'heroin',
              'cocaine', 'mdma', 'pill', 'cocaine', 'xtc', 'hash', 'cap'
              'lsd', 'crystal', 'speed', 'xanax', 'mg', 'syrup', 'bud',
              'gram', 'caps' , "oxycodone", 'mushroom', 'amphetamine',
              "fentanyl", "weed", "heroin", "fishscale", "crack", 'neurobion'
              "vape ice", "valium", "ml", "promethazine", "flake", 'dihydrocodeine'
              "snort", "kush", "meth", "coke", 'ambien', 'dmt', 'jintropin'
             "pharm", "strain", "purity", "tablet", "clonazolam", "heroin",
             "dope", "candy", "aaa", "skunk", "drug", "bar", 'opiode'
             ]

serv_bank = ['youtube', 'vpn', 'vcc', 'amazon', 'hbo', 'tor', 'account',
            'bank', 'card', 'custom', 'custum', 'cc', 'cvv', 'get', 'http'
            'password', 'dns', 'phone', 'template', 'kindle','porn',
            'facebook', 'address', 'email', 'domino', 'bitcoin', 'lifetime'
            'cash', 'network', 'wifi', 'bill', 'fake', 'how', 'paypal',
            'computer', 'trojan', 'sql', 'window', 'app', 'ddos', 'service'
             'rat', 'hack', 'anti', 'data', 'linux', 'key', 'online', 'best'
            'web', 'net', 'visa', 'dump', 'bin', 'website', 'android', 'us'
            'exploit', 'money', 'bot', 'netflix', 'malware', 'adsense',
            'adword', 'security', 'download', 'file', 'tutorial', 'free',
            'microsoft', 'firewall', 'bypass', 'create', 'disable', 'adobe',
            'dual', 'spoof', 'spyware', 'scam', 'information', 'hulu']

guns_bank = ['pistol', 'ammunition', 'ak47', 'silencer', 'ammo', 'melee',
            '9mm', '5.6mm', 'rpg','revolver', 'gun', '357', 'steel', 
            'pump', 'gun', 'millimeter', 'uzi', 'shotty', 'sawed-off',
            'glock', 'escrow', 'blade', 'stealth', 'magazine', 'mags',
            'knife', 'stun', 'handle', 'caliber', 'barrel', 'weapon',
            'luger', 'round', 'voltage', 'compact', 'nato', 'microtech'
            'carry', 'dusters', 'conceal', 'rifle', 'pulse', 'action']

fake_bank = ['â', 'case', 'material', 'watch', 'steel' ,' stainless',
             'information', 'diameter', 'brand', 'thickness', 'replica','band', 
            'color', 'size', 'wear', 'rolex', 'passport', 'listing', 'id',
            'leather', 'business', 'fake', 'scanned', 'free', 'authentic',
            'counterfeit', 'original', 'hustlegod', 'flashlight', 'shock',
            'sunglass', 'armani', 'vuitton', 'forged', 'password', 'document'] 

other_bank = ['info', 'other', 'ebook', 'guide']
kaggle_drugs = ["drug", "tobacco", "chemicals"]
kaggle_serv = ["services", "data", "btc", "jewelry"]
kaggle_guns = ["weapon"]
kaggle_fake = ["counterfeits", "forgeries"]

def categorize_text(text, drug_bank, hack_bank, guns_bank, fake_bank):
    """
        This is the function to categorize items into either:
            Drugs, Services, Guns/Weapons, Counterfeits or Misc.
        You will need the word banks, above to to make this function work.
    """
    
    drugs = [bool(re.search(d, str(text).lower())) for d in drug_bank]
    guns = [bool(re.search(g, str(text).lower())) for g in guns_bank]
    hacks = [bool(h in str(text).lower()) for h in hack_bank]
    fake = [bool(f in str(text).lower()) for f in fake_bank]
    other = [bool(o in str(text).lower()) for o in other_bank]
    if sum(other) > sum(drugs): 
        if sum(other) > sum(fake):
            if sum(other) > sum(hacks):
                if sum(other) >= sum(drugs):
                    return 'Misc'
                else:
                    return 'Drugs'
            else:
                return 'Services'
        else:
            return 'Counterfeits'
    elif sum(guns) > sum(drugs): 
        return 'Weapons'
    elif sum(fake) > sum(drugs): 
        return 'Counterfeits'
    elif sum(drugs) >= sum(hacks): 
        return 'Drugs'
    elif sum(hacks) > sum(guns): 
        return 'Services'
    else: 
        return 'Misc'

In [9]:
def get_vendor_category_counts(df, name):
    df['Category'] = df['description'].apply(lambda x: categorize_text(x, drug_bank, serv_bank, guns_bank, fake_bank))
    vc_df = pd.DataFrame(df['Category'].value_counts())
    vc_df['Vendor'] = name
    vc_df = vc_df.reset_index()
    vc_df.columns = ['Category', 'Count', 'Vendor']
    return vc_df

def vendor_counts(path):
    try:
        df = pd.read_csv(path, usecols = ['vendor_name', 'description'] )
        vendors = df['vendor_name'].unique()
        vc_df = get_vendor_category_counts(df[df['vendor_name'] == vendors[0]], vendors[0])
        for v in vendors[1:]:
            temp = get_vendor_category_counts(df[df['vendor_name'] == v], v)
            vc_df = pd.concat([vc_df, temp])

        vc_df['Date'] = path
        vc_df['Date'] = vc_df['Date'].str.replace("[^(\d+\-\d+\-\d+)]", "")
        vc_df['Date'] = pd.to_datetime(vc_df['Date'])
        return vc_df
    except:
        df = pd.DataFrame(columns=['Category', 'Count', 'Vendor', 'Date'])
        return df

In [11]:
start = time.time()
agora_df = vendor_counts(agora_files[0])
for file in agora_files[1:]:
    temp = vendor_counts(file)
    agora_df = pd.concat([agora_df, temp])
end = time.time()
print(end - start)

/Users/Andy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



16479.681746959686


In [17]:
agora_df.shape

(293094, 4)

In [23]:
export_csv = agora_df.to_csv('../data/vendor_categories.csv')

## Data Cleaning and Analysis

In [3]:
agora_df = pd.read_csv('../data/vendor_categories.csv')

In [8]:
def kaggle_agora_df(path):
    
    '''This function is specific to the Kaggle Agora data. Thus, it takes in a file path to the Kaggle data and returns
    the dataset in the form of a dataframe with the correct data types for each column.'''

    df = pd.read_excel(path)
    
    # Readjust the columns
    df.columns = ['Vendor', 'Category', 'Item', 'Item Description', 'Price', 'Origin', 'Destination', 'Rating', 'Remarks']
    df = df.drop(df.columns[-1], axis = 1)
    
    df = df[['Category', 'Vendor', 'Item', 'Item Description', 'Price', 'Origin', 'Destination', 'Rating']]

    # Resolve issues due to shifted columns
    df['Origin'].fillna('', inplace = True)
    df.loc[df['Origin'].str.contains('BTC')] = 'None'
    
    # Strip meaningless characters in 'Origin' column
    df.loc[df['Origin'].str.contains('\)'), 'Origin'] = 'None'
    df.Origin.where(~df.Origin.str.contains('Undeclared'), 'Undeclared', inplace = True)

    # there are occasional ratings of [0 deals] which indicates insufficient deals to provide a concrete rating
    # replace strings containing 'deals' with NaN
    df['Rating'].fillna('', inplace = True)
    df.Rating.where(~df.Rating.str.contains('deals'), np.nan, inplace = True)
    
    # Strip unecessary characters in order to convert columns to appropriate type
    rate = pd.DataFrame(df['Rating'])
    df['Rating'] = rate['Rating'].str.replace("/5|[^(\d{1}\.\d{2-3})$]", "")
    df['Rating'] = pd.to_numeric(df['Rating'])

    price = pd.DataFrame(df['Price'])
    df['Price'] = price['Price'].str.replace("[^(\d+\.\d+)]", "")
    df.loc[df['Price'] == ''] = np.nan
    df['Price'] = pd.to_numeric(df['Price'])

    df = df[~df.duplicated(keep = False)]
    df = df.reset_index(drop = True)

    df = df.replace('', np.nan, regex=True)
    df = df.replace('None', None, regex=True)
    
    return df

path = '../data/dark-net-marketplace-drug-data-agora-20142015/Agora.xlsx'
agora = kaggle_agora_df(path)
agora.head()

,Category,Vendor,Item,Item Description,Price,Origin,Destination,Rating
0,Services/Hacking,CheapPayTV,12 Month HuluPlus gift Code,12-Month HuluPlus Codes for $25. They are wort...,0.050270,Torland,NaN,4.96
1,Services/Hacking,CheapPayTV,Pay TV Sky UK Sky Germany HD TV and much mor...,Hi we offer a World Wide CCcam Service for En...,0.152420,Torland,NaN,4.96
2,Services/Hacking,KryptykOG,OFFICIAL Account Creator Extreme 4.2,Tagged Submission Fix Bebo Submission Fix Adju...,0.007000,Torland,NaN,4.93
3,Services/Hacking,cyberzen,VPN > TOR > SOCK TUTORIAL,How to setup a VPN > TOR > SOCK super safe enc...,0.019017,NaN,NaN,4.89
4,Services/Hacking,businessdude,Facebook hacking guide,. This guide will teach you how to hack Faceb...,0.062018,Torland,NaN,4.88


In [3]:
# add more sub-categories to dataframe
agora['Category'] = agora['Category'].astype(str)
cat = [a[0] for a in agora['Category'].str.split("/")]
subcat = [w[1] if len(w) > 1 else w[0] + ": Misc" for w in agora['Category'].str.split("/")]
subsubcat = [w[2] if len(w) > 2 else  w[1] + ": Misc" if len(w) > 1 else w[0] + ": Misc" for w in agora['Category'].str.split("/")] 

d = { 
    'Vendor': agora['Vendor'],
    'Category': cat,
    'Sub-Category': subcat,
    'Sub-Sub-Category': subsubcat,
    'Item': agora['Item'],
    'Item Description': agora['Item Description'],
    'Price': agora['Price'],
    'Rating': agora['Rating']
}

agora_sub_cat = pd.DataFrame(data = d)

agora_sub_cat['Category'].unique()


array(['Services', 'Drugs', 'Forgeries', 'Tobacco', 'Counterfeits',
       'Data', 'Information', 'Electronics', 'Drug paraphernalia',
       'Other', 'Jewelry',
       'the Body Bags  shotty  and Mac make-up To: Kind Hearts and Gentle People',
       'Weapons', 'Info',
       'home of the Body Bags  shotty  and mag  Kind Hearts and Gentle People',
       'Chemicals', '0.12780125125 BTC', '0.1905617980645162 BTC',
       'the Body Bags  shotty  and mag To: Kind Hearts and Gentle People'],
      dtype=object)

In [6]:
# Apply categorization algorithm 
# Simplifies certain categories such as:
#   Tobacco, Drugs, and Drug Paraphenalia all gets lumped into Drugs.
agora_sub_cat['Category'] = agora_sub_cat['Category'].apply(
    lambda x: categorize_text(x, kaggle_drugs, kaggle_serv, kaggle_guns, kaggle_fake)
)

array(['Services', 'Drugs', 'Counterfeits', 'Misc', 'Weapons'],
      dtype=object)

In [10]:
vendors = agora['Vendor'].unique()

In [29]:
# value counts returned a list of the size 3191 which wouldn't let it merge with vnedors_list
posts = pd.DataFrame(agora['Vendor'].value_counts())['Vendor']
posts = [len(agora[agora['Vendor'] == vendor]) for vendor in vendors]

In [15]:
vendor_ratings = [np.mean(agora[agora['Vendor'] == vendor]['Rating']) for vendor in vendors]

In [38]:
vendors_df = pd.DataFrame({'Vendor':vendors, 'Avg Rating':vendor_ratings, 'Posts': posts})
topseller_list= vendors_df.sort_values(by = ['Posts', 'Avg Rating'], ascending = False)[:20]['Vendor']

top20 = agora_sub_cat[agora_sub_cat.Vendor.isin(topseller_list)]
notTop = agora_sub_cat[~agora_sub_cat.Vendor.isin(topseller_list)]
top20["Sub-Category"].value_counts().head()

eBooks          1469
Cannabis        1365
Prescription    1030
Ecstasy          941
Watches          818
Name: Sub-Category, dtype: int64

In [46]:
# get the post counts for each sub-category for the top 20 vendors
top20_counts = pd.DataFrame(top20['Sub-Category'].value_counts())
top20_counts = top20_counts.reset_index()
top20_counts.columns = ['Sub-Category', 'Count']

top20_df = pd.merge(top20_counts, top20, on = 'Sub-Category')
top20_df['Vendor'] = 'Top 20 Vendor'

# get the post counts for each sub-category for other vendors
notTop_counts = pd.DataFrame(notTop['Sub-Category'].value_counts())
notTop_counts = notTop_counts.reset_index()
notTop_counts.columns = ['Sub-Category', 'Count']

notTop_df = pd.merge(notTop_counts, notTop, on = 'Sub-Category')
notTop_df['Vendor'] = 'Rest of Vendors'

# get only the following columns: "Sub-Category", "Category", "Vendor", "Count"
df = top20_df.iloc[:, [0,1, 2, 3]]
df2 = notTop_df.iloc[:, [0,1, 2, 3]]

# combine the dataframes
comb_df = pd.concat([df, df2])

# drop duplicates
comb_df = comb_df.drop_duplicates()

#get the top 3 highest count of posts for each category
comb_df = comb_df.sort_values('Count', ascending = False).groupby(['Vendor', 'Category']).head(3)
comb_df = comb_df.reset_index(drop = True)

comb_df.sort_values(["Vendor", "Category"])
comb_df['Comb-Category'] = [comb_df['Category'][i] + ": " + comb_df['Sub-Category'][i] for i in range(0, len(comb_df['Category']))]
comb_df.head()

,Sub-Category,Count,Vendor,Category,Comb-Category
0,Cannabis,29796,Rest of Vendors,Drugs,Drugs: Cannabis
1,Ecstasy,14042,Rest of Vendors,Drugs,Drugs: Ecstasy
2,Stimulants,12655,Rest of Vendors,Drugs,Drugs: Stimulants
3,Money,1661,Rest of Vendors,Counterfeits,Counterfeits: Money
4,Money,1661,Rest of Vendors,Services,Services: Money


## Data Visualization

In [41]:
def vendor_trace(df, vendor):
    
    '''
    
    This function was made to generalize the creation of
    traces since  the code to make them is almost identical
    between traces.
    
    This function takes in a dataframe with a Comb-Category
    Vendor, and Count column. It then creates a plot.ly trace 
    for the type of vendor and returns said trace.
    
    '''
    
    temp_df = df[df['Vendor'] == vendor]
    trace = go.Bar(
        x = temp_df['Comb-Category'],
        y = temp_df['Count'],
        name = vendor
    )
    return trace

traces = [vendor_trace(comb_df, v) for v in comb_df['Vendor'].unique()]

layout = go.Layout(
    barmode='stack',
    
    title = 'Top 20 Vendor vs Rest of Vendors Activity in Different Categories', 
    
    xaxis=dict(
        title= 'Categories and Sub-Categories',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            color='black'
        ),
        
        showticklabels=True,
        tickangle=25,
        tickfont=dict(
            size= 10
        )
    ),
    
    yaxis = dict(
        title = 'Number of Postings',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            color='black'
        ),
        
        showticklabels=True,
    )
)

fig = go.Figure(data=traces, layout=layout)
py.iplot(fig, filename='top20-stacked-bar')
# plotly.offline.plot(fig, filename='top20vsrest.html')

/Users/Andy/anaconda3/lib/python3.7/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



[Link to Graph](https://plot.ly/~aialba/14/top-20-vendor-vs-rest-of-vendors-activity-in-different-categories/)

In [44]:
def sub_cat_df(cat, df):
    
    '''
        
    The intent of this function is to get the counts
    of observations for each sub-category within a 
    given category.
    
    This function takes in a category name and
    a dataframe with columns: Category and 
    Sub-Category. It then takes the counts of
    all of the Sub-Categories using value_counts(),
    formats the result as a df, adds a column
    for the initial category, and outputs the result.
    
    '''
    
    cat_df = pd.DataFrame(df[df['Category'] == cat]['Sub-Category'].value_counts())
    cat_df = cat_df.reset_index()
    cat_df.columns = ['Sub Category', 'Count']
    cat_df['Category'] = cat
    return cat_df

categories = agora_sub_cat['Category'].unique()
sub_cats = sub_cat_df(categories[0], agora_sub_cat)

for i in categories[1:len(categories)]:
    sub_cats = pd.concat( [sub_cats, sub_cat_df(i, agora_sub_cat)])
    
sub_cats = sub_cats.reset_index(drop = True)

sub_cats = sub_cats.drop([9, 10, 35, 36 ,37]) # dropping really unhelpful Sub categories
sub_cats.head()

,Sub Category,Count,Category
0,Money,1481,Services
1,Accounts,1233,Services
2,Pirated,529,Services
3,Other,487,Services
4,Hacking,453,Services


In [45]:
def subcategory_trace(df, category):
    
    '''   
    This function takes in a dataframe with a Comb-Category
    Vendor, and Count column. It then creates a plot.ly trace 
    for the type of vendor and returns said trace.
    
    This function was made to generalize the creation of
    traces since  the code to make them is almost identical
    between traces.
    
    ''' 
    
    trace = go.Bar(
        x = df[df['Category'] == category]['Sub Category'],
        y = df[df['Category'] == category]['Count'],
        name = category
    )
    return trace

traces = [subcategory_trace(sub_cats, c) for c in sub_cats['Category'].unique()]
layout = go.Layout(
    barmode='group',
    title = 'Number of Items Postings by Sub-Category', 
    
    xaxis=dict(
        title='Sub-Categories',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            color='black'
        ),
        
        showticklabels=True,
        tickangle=30,
        tickfont=dict(
            size= 10
        )
    ),
    
    yaxis = dict(
        title = 'Number of Postings',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            color='black'
        ),
        
        showticklabels=True,
    )
)

fig = go.Figure(data=traces, layout=layout)
py.iplot(fig, filename='top20-grouped-bar')
# plotly.offline.plot(fig, filename='subcatitems.html')

/Users/Andy/anaconda3/lib/python3.7/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



[Link to Graph](https://plot.ly/~aialba/16/number-of-items-postings-by-sub-category/)

I looked into the subcategories of what vendors on the Darknet Market Place Agora were selling. We can see that the most popular item category in Agora is Cannabis with 30,000 observations in the Kaggle data set. The market for Cannabis in Agora is so huge that it dwarfs all others and has more than double postings than the next most popular category. We found that more than 95% of the postings for Cannabis were from non top rated, active vendors. <br>

Continuing off the last point, the popularity of Cannabis in top rated, active vendors and others tells us that many vendors are selling Cannabis. One cause of the variety of vendors posting about Cannabis may come from its legalization across a few different state in the US (Berke). People who may want to make some easy money buying and reselling weed to states where it is not legal. Regardless of the potential reasons behind the popularity of Cannabis amongst Agora users, we can see current the current laws of the prohibition of Cannabis is widely unpopular.<br>

Moving on, the second most popular category of items that top rated, active vendors sold were e-books with almost 1,500 thousand observations of this in the Kaggle data set. This tells us that one of the most sustainable markets in Agora is for ebooks. This may come from the ease in generating or providing eBooks. Another reason why successful vendors have longer lifespans in this market may be because law enforcement agencies may not be looking for people who illegally share books online. This makes it a lot safer and sustainable for vendors to thrive and survive in this market and may also be why top rated, active vendors tend not to sell in the drugs market in Agora.